In [2]:
import json
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [5]:
# Load data user
user_data_path = "../result/user_data.json"
with open(user_data_path, "r") as f:
    user_data = json.load(f)

# Ekstraksi fitur dan label
X = []
y = []

# Encoding untuk tingkat aktivitas
activity_map = {
    "sedentary": 0,
    "light": 1,
    "average": 2,
    "active": 3,
    "very_active": 4
}


In [6]:
for user in user_data:
    X.append([
        user["weight"],  # Berat badan
        user["tall"],    # Tinggi badan
        user["goal_weight"],  # Berat badan yang diinginkan
        activity_map[user["active"]],  # Aktivitas sebagai angka
        user["age"]      # Umur
    ])
    y.append([
        user["target_calories"],
        user["target_carbs"],
        user["target_fats"],
        user["target_proteins"]
    ])

X = np.array(X)
y = np.array(y)

In [34]:
# Split data menjadi train dan test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalisasi data input (X)
scaler_X = StandardScaler()
X = scaler_X.fit_transform(X)

# Normalisasi target output (y)
scaler_y = MinMaxScaler()
y = scaler_y.fit_transform(y)
# Bangun model TensorFlow
def build_pfc_model(input_dim):
    model = Sequential([
        Dense(256, activation="relu", input_dim=input_dim),
        Dropout(0.3),
        Dense(128, activation="relu"),
        Dropout(0.2),
        Dense(64, activation="relu"),
        Dense(4)  # Output: target_calories, carbs, fats, proteins
    ])
    model.compile(optimizer="adam", loss="mse", metrics=["mae"])
    return model

In [38]:
# Inisialisasi model
model = build_pfc_model(X_train.shape[1])
early_stopping = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)
# Latih model
model.fit(X_train, y_train, epochs=5000, batch_size=16, validation_data=(X_test, y_test))

# Simpan model
model.save("result/model/target_pfc_model.h5")
print("Model berhasil disimpan ke result/model/target_pfc_model.h5")

Epoch 1/5000
50/50 [==============================] - 2s 11ms/step - loss: 826070.2500 - mae: 462.4137 - val_loss: 114497.2578 - val_mae: 185.9440
Epoch 2/5000
50/50 [==============================] - 0s 5ms/step - loss: 124137.9062 - mae: 187.0869 - val_loss: 105089.5781 - val_mae: 162.6994
Epoch 3/5000
50/50 [==============================] - 0s 6ms/step - loss: 118165.2812 - mae: 179.7018 - val_loss: 99701.2969 - val_mae: 157.1553
Epoch 4/5000
50/50 [==============================] - 0s 6ms/step - loss: 116446.0078 - mae: 180.5546 - val_loss: 95503.3906 - val_mae: 155.0806
Epoch 5/5000
50/50 [==============================] - 0s 6ms/step - loss: 103658.8203 - mae: 169.2088 - val_loss: 93899.5625 - val_mae: 156.2053
Epoch 6/5000
50/50 [==============================] - 0s 6ms/step - loss: 105008.8125 - mae: 171.2870 - val_loss: 87917.5781 - val_mae: 146.8367
Epoch 7/5000
50/50 [==============================] - 0s 6ms/step - loss: 103689.0000 - mae: 168.6392 - val_loss: 83714.6328 - 

In [ ]:
# Load model
from tensorflow.keras.models import load_model
model = load_model("result/model/target_pfc_model.h5")

# Contoh pengguna untuk prediksi
new_user = np.array([
    [91.0, 165.0, 110.0, activity_map["average"], 48]  # Data pengguna baru
])
#bb,tinggi, target berat, activity, age
# Prediksi
predicted_pfc = model.predict(new_user)
print("Prediksi PFC:")
print(f"Target Calories: {predicted_pfc[0][0]:.2f}")
print(f"Target Carbs: {predicted_pfc[0][1]:.2f}")
print(f"Target Fats: {predicted_pfc[0][2]:.2f}")
print(f"Target Proteins: {predicted_pfc[0][3]:.2f}")


1/1 [==============================] - 0s 118ms/step
Prediksi PFC:
Target Calories: 2154.22
Target Carbs: 250.34
Target Fats: 62.78
Target Proteins: 170.59
